# Paper Functions
Nathan DeBardeleben, ndebard@lanl.gov, HPC-DES, ~March, 2021

These are Python functions which produce the plots for the SC2021 LANL/CCU statistics paper on acceptance testing.

In [ ]:
from scipy.stats import poisson
from scipy.stats import gamma
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.optimize import fmin
from numpy import array
import pandas as pd
import seaborn as sns
from scipy.optimize import minimize

In [ ]:
def pff_calculator(x, F, pff):
    # in R
    # abs(1-ppois(q=q,lambda=x)-pff)
    return abs(1 - poisson.cdf(F, x) - pff)
def one_minus_poisson_cfd(x, F):
    return abs(1 - poisson.cdf(F, x))
def pfp_calculator(x, F, pfp):
    # in R
    # abs(ppois(q=q,lambda=x)-pfp)
    return abs(poisson.cdf(F, x) - pfp)
def poisson_cfd(x, F):
    return abs(poisson.cdf(F, x))
def two_party_optimization(x, F, factor, pfp, pff):
    # in R
    # abs(1-ppois(q=quse,lambda=x)-pff) + abs(ppois(q=quse,lambda=fac*x)-pfp)
    return ( abs(1 - poisson.cdf(F, x) - pff) + abs(poisson.cdf(F, x * factor) - pfp) )

# Plotting Parameters
Here we define some parameters for plotting.

# Parameters of Interest
Here we set our initial constraints.

In [ ]:
pff = 0.1 # probability of false failure
pfp = 0.1 # probability of false pass
F = 1 # failures
mu = 24 # hours
factor = 2

# Probability of False Fail
This is related to the vendor (producer) failing a test that they should have passed.  They want to minimize that happening.

## Find the Min Theta
First, we need to find the min theta for these above constraints.

In [ ]:
func = pff_calculator
ret = fmin(func, F/2, args=(F, pff), xtol = 0.00001, full_output=True)
the_min_x = ret[0][0]
the_y_val = ret[1]
print(f"The min of function '{func.__name__}' is at {the_min_x} with result {the_y_val}.")

Set up our range of x values, based on what we know the min is - this is just for pretty plotting.

In [ ]:
xmin = 0
xmax = math.floor(the_min_x) * 2
if xmax < 5:
    xmax = 5
points = 10000
# generate x points
xlist = np.linspace(xmin, xmax, points)

### Plot the Min
This plots the function and annotates the min

In [ ]:
func = pff_calculator

# map the x points to the values from the above function
ylist = list(map(lambda x: func(x, F=F, pff=pff), xlist))

# plot the results, w/ annotations
plt.plot(xlist, ylist)

# # draw horizontal line at min of function
plt.hlines(y=the_y_val, xmin=min(xlist), xmax=max(xlist), color='red', linestyles="--")
plt.annotate(f"function min = {the_y_val:.3f}", xy=(2, the_y_val + max(ylist) * 0.05), color='red')

# draw vertical line at min
plt.vlines(x=the_min_x, ymin=min(ylist), ymax=max(ylist), color='green', linestyles='--')
plt.annotate(f"θ = {the_min_x:.3f}", xy=(the_min_x + max(xlist) * 0.025, 0.5), color='green')

# labels and title
plt.xlabel("θ")
plt.ylabel(f"{func.__name__} output")
plt.title(f"Calculating output of {func.__name__} for Varying θ and Fixed F={F}")
plt.savefig(f"NOT_USED_IN_PAPER_pff_calculator_f{F}_pff{pff}.png")
plt.savefig(f"NOT_USED_IN_PAPER_pff_calculator_f{F}_pff{pff}.pdf")

### Show the Min With Constraints
Going back to our initial distribution, show this point crossing the constraint

In [ ]:
func = one_minus_poisson_cfd

# map the x points to the values from the above function
ylist = list(map(lambda x: func(x, F), xlist))

In [ ]:
df = pd.DataFrame()
df['x'] = xlist
df['pff'] = ylist
df.columns = ['theta', '$P_{FF}$']
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax = sns.lineplot(data=df, x='theta', y='$P_{FF}$')
ax.axhline(pfp, ls='--', color='green')
ax.annotate(text=f"θ = {the_min_x:.3f}", 
            xy=(the_min_x, the_y_val + pfp), 
            xycoords='data', 
            xytext=(10, 100), 
            fontsize=12,
            textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.1)
           )
ax.plot([the_min_x],[the_y_val + pfp],'ro', mec='black', color='red', ms=10, linewidth=5)
plt.xlabel("θ: Multiple of Test Duration, $\mu$") # matplotlib doesn't like $\theta$ for some reason, wtf?
plt.ylabel("$P_{FF}$: Probability of a False Fail")
plt.title("Probability of a False Fail for Varying θ and Fixed F=1")
fn = f"FIGURE_2_prob_false_fail_{F}_pff{pff}"
plt.savefig(f"{fn}.pdf")
plt.savefig(f"{fn}.png")
print(f"{fn}.png and .pdf created")

## Result
These are the end results for this section:

In [ ]:
print(f"The vendor / producer:\n" \
      f"  To minimize false failure, with acceptable probability of false failure = {pff}\n" \
      f"  With mu (MTBF) of {mu} (hours)\n" \
      f"  And testing until <= {F} failures are observed\n" \
      f"  Wants to test for {the_min_x:.3f} * {mu} or {(the_min_x * mu):.3f} hours")

# Probability of False Pass
This is related to the buyer (consumer) failing to reject a system that is actually bad.  They want to minimize that happening.

## Find the Min Theta
First, we need to find the min theta for these above constraints.

In [ ]:
func = pfp_calculator
ret = fmin(func, F/2, args=(F, pfp), xtol = 0.00001, full_output=True)
the_min_x = ret[0][0]
the_y_val = ret[1]
print(f"The min of function '{func.__name__}' is at {the_min_x} with result {the_y_val}.")

Set up our range of x values, based on what we know the min is - this is just for pretty plotting.

In [ ]:
xmin = 0
xmax = math.floor(the_min_x) * 2
if xmax < 5:
    xmax = 5
points = 10000
# generate x points
xlist = np.linspace(xmin, xmax, points)

### Plot the Min
This plots the function and annotates the min

In [ ]:
func = pfp_calculator

# map the x points to the values from the above function
ylist = list(map(lambda x: func(x, F=F, pfp=pfp), xlist))

# plot the results, w/ annotations
plt.plot(xlist, ylist)

# # draw horizontal line at min of function
plt.hlines(y=the_y_val, xmin=min(xlist), xmax=max(xlist), color='red', linestyles="--")
plt.annotate(f"function min = {the_y_val:.3f}", xy=(2, the_y_val + max(ylist) * 0.05), color='red')

# draw vertical line at min
plt.vlines(x=the_min_x, ymin=min(ylist), ymax=max(ylist), color='green', linestyles='--')
plt.annotate(f"θ = {the_min_x:.3f}", xy=(the_min_x + max(xlist) * 0.025, 0.5), color='green')

# labels and title
plt.xlabel("θ")
plt.ylabel(f"{func.__name__} output")
plt.title(f"Calculating output of {func.__name__} for Varying θ and Fixed F={F}")
plt.savefig(f"NOT_USED_IN_PAPER_pfp_calculator_f{F}_pfp{pfp}.png")
plt.savefig(f"NOT_USED_IN_PAPER_pfp_calculator_f{F}_pfp{pfp}.pdf")

### Show the Min With Constraints
Going back to our initial distribution, show this point crossing the constraint

In [ ]:
func = poisson_cfd

# map the x points to the values from the above function
ylist = list(map(lambda x: func(x, F), xlist))

In [ ]:
df = pd.DataFrame()
df['x'] = xlist
df['pff'] = ylist
df.columns = ['theta', '$P_{FP}$']
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax = sns.lineplot(data=df, x='theta', y='$P_{FP}$')
ax.axhline(pfp, ls='--', color='green')
ax.annotate(text=f"θ = {the_min_x:.3f}", 
            xy=(the_min_x, the_y_val + pfp), 
            xycoords='data', 
            xytext=(50, 50), 
            fontsize=12,
            textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.1)
           )
ax.plot([the_min_x],[the_y_val + pfp],'ro', mec='black', color='red', ms=10, linewidth=5)
plt.xlabel("θ: Multiple of Test Duration, $\mu$") # matplotlib doesn't like $\theta$ for some reason, wtf?
plt.ylabel("$P_{FP}$: Probability of a False Pass")
plt.title("Probability of a False Pass for Varying θ and Fixed F=1")
fn = f"FIGURE_1_prob_false_pass_{F}_pfp{pfp}"
plt.savefig(f"{fn}.pdf")
plt.savefig(f"{fn}.png")
print(f"{fn}.png and .pdf created")

## Result
These are the end results for this section:

In [ ]:
print(f"The buyer / consumer:\n" \
      f"  To minimize false pass, with acceptable probability of false pass = {pfp}\n" \
      f"  With mu (MTBF) of {mu} (hours)\n" \
      f"  And testing until <= {F} failures are observed\n" \
      f"  Wants to test for {the_min_x:.3f} * {mu} or {(the_min_x * mu):.3f} hours")

# Constrain PFF and Match PFP with `factor` Tolerance
This code locks PFF at the input value and then plots PFP and the intersection of these curves

In [ ]:
# some helper code to find intersection of columns from the dataframe
def find_intersection_of_two_curves(x, y1, y2, deg):
    y1_fit = np.polyfit(x, y1, deg)
    y1_func = np.poly1d(y1_fit)

    y2_fit = np.polyfit(x, y2, deg)
    y2_func = np.poly1d(y2_fit)

    from scipy.optimize import fsolve
    def findIntersection(fun1,fun2,x0):
        return fsolve(lambda x : fun1(x) - fun2(x), x0)

    result = findIntersection(y1_func, y2_func, 0.0)
    return (result[0],y1_func(result[0]))

In [ ]:
def constrain_pff_find_pfp(pff, factor, title_prepend):
    # generate the dataframe with F, theta, and values of PFF, PFP (where PFP has a `factor` tolerance)
    d = dict()
    for Ftemp in range(0, 31):
        d[Ftemp] = list()
        d[Ftemp].append(Ftemp)
        res = minimize(fun=pff_calculator, args=(Ftemp, pff), x0=Ftemp/2, tol=0.0000001, method='Nelder-Mead')
        d[Ftemp].append(res['x'][0])
        d[Ftemp].append(one_minus_poisson_cfd(res['x'][0], Ftemp))
        d[Ftemp].append(poisson_cfd(res['x'] * factor, Ftemp)[0])

    df = pd.DataFrame.from_dict(d, orient='index', columns=['F','theta','$P_{FF}$','$P_{FP}$'])    
    intersection_x,intersection_y = find_intersection_of_two_curves(df['F'].to_numpy(), df['$P_{FF}$'].to_numpy(), df['$P_{FP}$'].to_numpy(), 10)
    print(f"These curves intersect at x={intersection_x}, y={intersection_y}")
    idx = df['F'].sub(intersection_x).abs().idxmin()
    theta_at_intersection = df.iloc[idx]['theta']
    print(f"Theta value at this intersection is {theta_at_intersection}")
    
    # melt (transform) the dataframe for plotting
    df2 = df.melt(id_vars=['F','theta'], var_name='Constraints', value_name='Probability')
    
    # and plot it
    fig, ax = plt.subplots(figsize=(6,4))
    ax = sns.lineplot(data=df2, x='F', y='Probability', hue='Constraints')
    ax.annotate(text=f"θ = {theta_at_intersection:.4f}\nF = {df.iloc[idx]['F']:.0f}", 
                xy=(intersection_x, intersection_y), 
                xycoords='data', 
                xytext=(50, 50), 
                fontsize=12,
                textcoords='offset points',
                arrowprops=dict(facecolor='black', shrink=0.1)
               )
    ax.plot([intersection_x],[intersection_y],'ro', mec='black', color='red', ms=10, linewidth=5)
    plt.xlabel("F: # of Allowable Failures During Test")
    plt.title("Constraining $P_{FF} = 0.1$, $\lambda_{bad} = %s\lambda_{good}$" % factor)    
    fn = f"{title_prepend}constrained_pff{pff}_W{factor}"
    plt.savefig(f"{fn}.png");
    plt.savefig(f"{fn}.pdf");
    print(f"{fn}.png and .pdf created")

In [ ]:
constrain_pff_find_pfp(pff=0.1, factor=2, title_prepend="FIGURE_3_")

In [ ]:
constrain_pff_find_pfp(pff=0.1, factor=5, title_prepend="NOT_USED_IN_PAPER_")

# Optimize to Both Parties' Goals
The previous section assumed that PFP would be interested in the same goal as PFF, here we allow them to vary

In [ ]:
def constraint_pff_and_pfp_optimize(pff, pfp, factor, Fmax, degree_poly, title_prepend):
    # generate the dataframe with F, theta, and values of PFF, PFP (where PFP has a `factor` tolerance)
    d = dict()
    for Ftemp in range(0, Fmax):
        d[Ftemp] = list()
        d[Ftemp].append(Ftemp)
        res = minimize(fun=two_party_optimization, args=(Ftemp, factor, pfp, pff), x0=Ftemp/2, tol=0.0000001, method='Nelder-Mead')
        d[Ftemp].append(res['x'][0])
        d[Ftemp].append(res['fun'])

    df = pd.DataFrame.from_dict(d, orient='index', columns=['F','theta','objective_function'])
    #display(df)
    
    obj_fit = np.polyfit(df['F'].to_numpy(), df['objective_function'].to_numpy(), degree_poly)
    obj_func = np.poly1d(obj_fit)
    min_res = minimize(obj_func, x0=0, method='Nelder-Mead')
    min_value = min_res['x'][0]
    print(min_value)
    idx = df['F'].sub(min_value).abs().idxmin()
    theta_at_min = df.iloc[idx]['theta']
    f_at_min = df.iloc[idx]['F']
    obj_at_min = df.iloc[idx]['objective_function']
    print(f"F value at this min is {f_at_min}")
    print(f"Theta value at this min is {theta_at_min}")
    print(f"Objective value at this min is {obj_at_min}")
    
    fig, ax = plt.subplots(figsize=(6,4))
    ax = sns.lineplot(data=df, x='F', y='objective_function')

    ax.annotate(text=f"θ = {theta_at_min:.4f}\nF = {f_at_min:.0f}", 
                xy=(f_at_min, obj_at_min), 
                xycoords='data', 
                xytext=(50, 100), 
                fontsize=12,
                textcoords='offset points',
                arrowprops=dict(facecolor='black', shrink=0.1)
               )
    ax.plot([f_at_min],[obj_at_min],'ro', mec='black', color='red', ms=10, linewidth=5)

    #plt.xticks(range(0, Fmax))
    plt.ylabel("Objective Function")
    plt.xlabel("F: # of Allowable Failures During Test")

    plt.title("Minimizing the Difference Between Objective Functions\n$P_{FF}$ = %s, $P_{FP}$ = %s, $\lambda_{bad} = %s\lambda_{good}$" % (pff, pfp, factor));
    fn = f"{title_prepend}two_party_pff{pff}_pfp{pfp}_W{factor}"
    plt.savefig(f"{fn}.png");
    plt.savefig(f"{fn}.pdf");
    print(f"{fn}.png and .pdf created")

In [ ]:
constraint_pff_and_pfp_optimize(pff=0.1, pfp=0.1, factor=2, Fmax=31, degree_poly=10, title_prepend="FIGURE_4_")

In [ ]:
constraint_pff_and_pfp_optimize(pff=0.1, pfp=0.1, factor=5, Fmax=6, degree_poly=4, title_prepend="FIGURE_7_")

# Constraining T

Try and fix T and see if we can optimize

In [ ]:
def constrained_time(T, W, title_prepend):
    T_hours = T * 24
    # generate the dataframe with F, theta, and values of PFF, PFP (where PFP has a `factor` tolerance)
    d = dict()
    for Ftemp in range(0, 31):
        d[Ftemp] = list()
        d[Ftemp].append(Ftemp)
        pff = 1 - poisson.cdf(Ftemp, T_hours / 24)
        pfp = poisson.cdf(Ftemp, W * T_hours / 24)    
        d[Ftemp].append(pff)
        d[Ftemp].append(pfp)

    df = pd.DataFrame.from_dict(d, orient='index', columns=['F','$P_{FF}$','$P_{FP}$'])
    df.head()
    df_melted = df.melt(id_vars=['F'])
    df_melted.head()
    fig, ax = plt.subplots(figsize=(6,4))
    ax = sns.lineplot(data=df_melted, x='F', y='value', hue='variable')
    plt.ylabel("Probability")
    plt.xlabel("F: # of Allowable Failures During Test")
    plt.title("$P_{FF}$ and $P_{FP}$ for a fixed $\lambda_{bad} = %s\lambda_{good}$\n" \
              f"For Fixed Test Duration = {T} days ({T_hours} hours)" % factor)

    plt.savefig(f"{title_prepend}fixed_time_interval_THours{T_hours}_F{F}_W{factor}.png");
    plt.savefig(f"{title_prepend}fixed_time_interval_THours{T_hours}_F{F}_W{factor}.pdf");

In [ ]:
constrained_time(T=7, W=2, title_prepend="FIGURE_5_")

In [ ]:
constrained_time(T=3, W=2, title_prepend="FIGURE_6_")

# Bayesian

## Confidence Intervals On Determinging MTBF

# <font color=red>THIS IS TABLE 2</font>

In [ ]:
def get_gamma_pi_95(alpha_prior, beta_prior, mu, X, T_Days):
    alpha_post = alpha_prior + X
    beta_post = beta_prior + 1
    T_Hours = T_Days * 24
    dist = gamma(alpha_post, scale=1.0/beta_post)
    # can do this if you want to sample it, without analytical results
    #random_variables = T_Hours / (dist.rvs(size=pow(10,7))) # pull 10^7 random variables
    #np.count_nonzero(random_variables > mu) / len(random_variables))))
    
    # PI and posterior prob that measured mu >= mu
    return (tuple((T_Hours / dist.ppf(0.975),
                   T_Hours / dist.ppf(0.025),
                   dist.cdf(T_Hours / mu))))

In [ ]:
mu = 24
T_Days = 7
alpha_prior = pow(10, -6)
beta_prior = pow(10, -4)

In [ ]:
get_gamma_pi_95(alpha_prior, beta_prior, mu, 1, 7)

In [ ]:
get_gamma_pi_95(alpha_prior, beta_prior, mu, 2, 7)

In [ ]:
get_gamma_pi_95(alpha_prior, beta_prior, mu, 3, 7)

In [ ]:
get_gamma_pi_95(alpha_prior, beta_prior, mu, 4, 7)

In [ ]:
get_gamma_pi_95(alpha_prior, beta_prior, mu, 5, 7)

In [ ]:
get_gamma_pi_95(alpha_prior, beta_prior, mu, 6, 7)

In [ ]:
get_gamma_pi_95(alpha_prior, beta_prior, mu, 7, 7)

In [ ]:
get_gamma_pi_95(alpha_prior, beta_prior, mu, 8, 7)

In [ ]:
get_gamma_pi_95(alpha_prior, beta_prior, mu, 9, 7)

In [ ]:
get_gamma_pi_95(alpha_prior, beta_prior, mu, 10, 7)